In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors3D
import pandas as pd
from Hyprid import polFeature
from rdkit.Chem import Descriptors

In [2]:
def hybConn(mol):
    sp3sp3 = 0
    sp3sp2 = 0
    sp3sp = 0
    sp2sp = 0
    sp2sp2 = 0
    spsp = 0
    sp3s = 0
    sp2s = 0
    sps = 0
    other = 0
    for bond in mol.GetBonds():
        atom1 = bond.GetBeginAtom()
        atom2 = bond.GetEndAtom()
        hyb1 = str(atom1.GetHybridization())
        hyb2 = str(atom2.GetHybridization())
        if   hyb1=="SP3" and hyb2=="SP3":sp3sp3+=1
        elif hyb1=="SP3" and hyb2=="SP2":sp3sp2+=1
        elif hyb1=="SP3" and hyb2=="SP" :sp3sp+=1
        elif hyb1=="SP3" and hyb2=="S"  :sp3s+=1
        elif hyb1=="S"   and hyb2=="SP3":sp3s+=1
        elif hyb1=="SP2" and hyb2=="SP3":sp3sp2+=1
        elif hyb1=="SP2" and hyb2=="SP2":sp2sp2+=1
        elif hyb1=="SP2" and hyb2=="SP" :sp2sp+=1
        elif hyb1=="SP2" and hyb2=="S"  :sp2s+=1
        elif hyb1=="S"   and hyb2=="SP2":sp2s+=1
        elif hyb1=="SP"  and hyb2=="SP3":sp3sp+=1
        elif hyb1=="SP"  and hyb2=="SP2":sp2sp+=1
        elif hyb1=="SP"  and hyb2=="SP" :spsp+=1
        elif hyb1=="SP"  and hyb2=="S"  :sps+=1
        elif hyb1=="S"   and hyb2=="SP" :sps+=1
        else:other+=1
    return [sp3sp3,sp3sp2,sp3sp,sp3s,sp2sp2,sp2sp,sp2s,spsp,sps,other]

In [3]:
def getMolDescriptors(mol):
    res = []
    for nm,fn in Descriptors._descList:
        if nm != 'Ipc':
            try:
                val = fn(mol)
                res.append(val)
            except:
                res.append(None)
                print(nm)
    return res

In [ ]:
# Load and preprocess data
filename = "../Raw Files/bindingdb_G12C.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename,sep='\t')
df.dropna(inplace=True)
hybd_names = ['FC','N','H','Cl','Br',
                 'I','P','F','Se','Si',
                 'S_sp3','S_sp2','S_sp',
                 'C_sp3','C_sp2','C_sp',
                 'N_sp3','N_sp2','N_sp',
                 'O_sp3','O_sp2','O_sp',
                 'I1','I2','I3']
bondhybd_names = ['sp3sp3','sp3sp2','sp3sp','sp3s',
                  'sp2sp2','sp2sp','sp2s','spsp','sps','other']
count = 0
dcr_names = [nm for nm,fn in Descriptors._descList]
dcr_names.remove('Ipc')
Cnames = ["ChEMBL ID","Smiles"]+hybd_names+bondhybd_names+dcr_names
print("Cnames", len(Cnames))
DF = pd.DataFrame(columns=Cnames)

In [4]:
filename = "../Raw Files/bindingdb_G12C.tsv"
#df = pd.read_csv("fda_original.csv",sep=";")
df = pd.read_csv(filename, sep='\t')
df.dropna(subset=['IC50 (nM)'], inplace=True)
df

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain.1,UniProt (SwissProt) Entry Name of Target Chain.1,UniProt (SwissProt) Primary ID of Target Chain.1,UniProt (SwissProt) Secondary ID(s) of Target Chain.1,UniProt (SwissProt) Alternative ID(s) of Target Chain.1,UniProt (TrEMBL) Submitted Name of Target Chain.1,UniProt (TrEMBL) Entry Name of Target Chain.1,UniProt (TrEMBL) Primary ID of Target Chain.1,UniProt (TrEMBL) Secondary ID(s) of Target Chain.1,UniProt (TrEMBL) Alternative ID(s) of Target Chain.1
0,1432260,COc1nc2nc(C)nc(N[C@H](C)c3cccc(c3C)C(F)(F)F)c2...,InChI=1S/C25H29F3N5O3P/c1-14-18(7-6-8-20(14)25...,ORHCSIAHTHKJHS-OAHLLOKOSA-N,688031,"US20240247015, Example 144",Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,1.97,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
1,1432311,C[C@@H](Nc1nc(Cl)nc2cnc(cc12)P1(=O)CCN(CC1)C(C...,InChI=1S/C23H24ClF3N5O2P/c1-13-16(5-4-6-18(13)...,YYURWWQWHPBABQ-CQSZACIVSA-N,688082,"US20240247015, Example 196",Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,2.38,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
2,1348122,C[C@@H](Nc1nc(C)nc2ncc(cc12)N1CCN(CC1=O)C(C)=O...,InChI=1S/C24H25F3N6O2/c1-13-18(6-5-7-20(13)24(...,NGPCTJVJEYPIMD-CQSZACIVSA-N,634093,4-acetyl-1-[2-methyl-4-({(1R)-1-[2-methyl-3-(t...,Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,2.53,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
3,1348107,C[C@@H](Nc1nc(C)nc2ncc(cc12)C1=CCN(CC1)C(C)=O)...,InChI=1S/C27H30F3N5O2/c1-15(20-7-6-8-22(23(20)...,UIPMZLHNJRPRAZ-OAHLLOKOSA-N,634078,"1-{4-[4-({(1R)-1-[3-(1,1-difluoro-2-hydroxy-2-...",Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,2.70,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
4,1348085,C[C@@H](Nc1nc(C)nc2nc(c(cc12)N1CC2(CN(C2)C(C)=...,InChI=1S/C26H26F6N6O2/c1-13(16-5-4-6-18(20(16)...,JNMKLENQHMUTDV-CYBMUJFWSA-N,634056,"1-{6-[4-({(1R)-1-[3-(1,1-difluoro-2-hydroxyeth...",Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,2.80,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,1386298,COc1cc2nc(C)nc(NC(C)c3ccc(s3)-c3ccc(F)cc3CN)c2...,InChI=1S/C24H25FN4O2S/c1-13(22-7-8-23(32-22)17...,GKYVQKWLGKKVDV-UHFFFAOYSA-N,657555,N-[1-{5-[2-(aminomethyl)-4-fluorophenyl]thioph...,Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,92.1,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
1851,1438494,C[C@@H](Nc1nn(C)c(=O)c2cc(=O)n(cc12)[C@H]1C[C@...,InChI=1S/C22H20F3N5O2/c1-11(14-4-3-5-15(19(14)...,LGZGCCYDIVGCQG-JHJVBQTASA-N,692087,"(1S,3s)-3-(4-(((R)-1-(3-(difluoromethyl)-2-flu...",Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,92.8,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
1852,1432129,C[C@@H](Nc1nc(C)nc2cnc(cc12)P1(=O)CCN(Cc2ccccc...,InChI=1S/C28H29F3N5OP/c1-18(21-9-6-10-22(26(21...,UOIMNMAYBXBYDT-GOSISDBHSA-N,687900,1-benzyl-4-[4-({(1R)-1-[3-(difluoromethyl)-2-f...,Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,94.0,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN
1853,1386990,CC(Nc1nc(C)nc2ccc(cc12)N1CCCC1)c1ccc(s1)-c1ccc...,InChI=1S/C28H33N5S/c1-19(26-13-14-27(34-26)23-...,ZEBUQLUWVPYOPQ-UHFFFAOYSA-N,657735,N-[1-(5-{2-[(dimethylamino)methyl]phenyl}thiop...,Son of sevenless homolog 1/GTPase KRas [G12C],Homo sapiens,NaN,94.2,...,Son of sevenless homolog 1,SOS1_HUMAN,Q07889,A8K2G3 B4DXG2,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Iterate through ChEMBL IDs and SMILES
count = 0
for i, (chembl_id, smiles) in enumerate(zip(df["BindingDB Reactant_set_id"], df["Ligand SMILES"])):
    try:
        count += 1
        if count == 100:
            count = 0
            print("Processing molecule number", i)

        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            print(f"Skipping {chembl_id}: Invalid SMILES '{smiles}'")
            continue

        mol = Chem.AddHs(mol)  # Add hydrogens
        status = AllChem.EmbedMolecule(mol, AllChem.ETKDG())

        if status == 0:
            t = getMolDescriptors(mol)
            hyp = polFeature(mol)
            bhyp = hybConn(mol)
            I = [Descriptors3D.PMI1(mol), Descriptors3D.PMI2(mol), Descriptors3D.PMI3(mol)]

            DF.loc[i] = [chembl_id, smiles] + hyp + bhyp + I + t
            print(f"Processed {chembl_id} successfully")
        else:
            print(f"Skipping {chembl_id}: 3D conformer generation failed")

    except Exception as e:
        print(f"Error processing {chembl_id} ({smiles}): {repr(e)}")

# Save to CSV
output_filename = "g12c_Hyb_Features.csv"
DF.to_csv(output_filename, index=False)
print(f"Saved output to {output_filename}")

In [ ]:
import pandas as pd

# Load the files
df_ic50 = pd.read_csv("../Raw Files/bindingdb_G12C.tsv", sep="\t")  # Read TSV file
df_features = pd.read_csv("g12c_Hyb_Features.csv")

# Merge on the respective ID columns
merged_df = df_features.merge(df_ic50[['BindingDB Reactant_set_id', 'IC50 (nM)']],
                              left_on="ChEMBL ID",
                              right_on="BindingDB Reactant_set_id",
                              how="left")

# Drop the duplicate ID column if needed
merged_df.drop(columns=["BindingDB Reactant_set_id"], inplace=True)

# Save the merged file
merged_df.to_csv("G12C_training.csv", index=False)

print("Merge completed. File saved as merged_features_IC50_g12c.csv")